# Lets import some things

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms


# Decide if cuda

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Load dataset

In [72]:
trainingData = datasets.Flowers102(
    root="data",
    split = "train",
    download = True,
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)
testData = datasets.Flowers102(
    root="data",
    split= "test",
    download = True,
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)
valData = datasets.Flowers102(
    root="data",
    split = "val",
    download = True,
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
)

# Get some dataloaders

In [73]:
batchsize = 32
train_dataloader = DataLoader(trainingData, batch_size=batchsize, shuffle=True, num_workers=2)
test_dataloader = DataLoader(testData, batch_size=batchsize, shuffle=True,num_workers=2)
val_dataloader = DataLoader(valData, batch_size=batchsize, shuffle=True,num_workers=2)

#todo get some dataloaders of the form compose

# Neural Network class

In [74]:
class NeuralNet(nn.Module):
    def __init__(self,num_classes):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1 ,padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,32,kernel_size=5,stride=1,padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(32,120)
        self.relu= nn.ReLU()
        self.fc1 = nn.Linear(120,num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out= self.layer2(out)
        out = out.reshape(out.size(0),-1)
        out = self.fc(out)
        out= self.relu(out)
        out= self.fc1(out)
        return out
#conv way selected, this is a uhhh LeNet5 for MNIST so it doesnt work like yeah of course pretty placeholder

In [75]:
batchsize = 64
num_classes = 102
learning_rate = 0.001
num_epochs = 3

# todo train/test/possibly do the image display thing/ save&load models/ https://pytorch.org/tutorials/beginner/introyt/introyt1_tutorial.html / hyperparams

# Model = something

In [76]:
model = NeuralNet(num_classes).to(device)
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
total_step = len(train_dataloader)

In [77]:
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = cost(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 400 == 0:
            print('Epoch [{},{}], Step [{},{}], Loss: {:.4f}'.format(epoch+1, 3, i+1, total_step, loss.item()))
# kinda placeholding also

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x89888 and 32x120)